In [14]:
import pandas as pd
import numpy as np
import requests
import datetime

In [15]:
# This will be the function that pulls the rolling data. Will pull the past 8 months
def pull_data(df):
    today = datetime.datetime.now()

    for i in range(4, 252, 3):
        date_end = datetime.date.today()
        delta_end = datetime.timedelta(days=i)
        delta_start = datetime.timedelta(days=i+3)
        date_end = date_end - delta_end
        date_start = today - delta_start

        date_end = date_end.strftime("%Y-%m-%d")
        date_start = date_start.strftime("%Y-%m-%d")
        response = requests.get(f"http://144.39.204.242:11236/evr/leviton/evr?dateStart={date_start}&dateEnd={date_end}")
        usage = response.json()

        ten_minute_averages = {"timestamp": [], "power": []}

        for i in range(0, len(usage['data']), 85):
            values = usage['data'][i:i+120]
            power = [value['power'] for value in values]
            ten_minute_averages["timestamp"].append(values[0]['timestamp'])
            ten_minute_averages["power"].append(np.average(power) * 1000)
        
        temp_df = pd.DataFrame(ten_minute_averages)
        df = pd.concat([df, temp_df], ignore_index=True)
    
    
    return df

In [16]:
df = pd.DataFrame(columns=["timestamp", "power"])
df= pull_data(df)
df.tail()
df.to_csv("power_usage.csv")